# Two-Child Limit Policy Parameters

## Overview

The two-child limit is a UK welfare policy that restricts child-related benefits to the first two children in a family for children born after April 2017. This policy affects both Universal Credit (UC) and Child Tax Credit (CTC).

PolicyEngine allows modeling both the current implementation and policy reforms to the two-child limit, including an age-based exemption mechanism.

## Current Policy Implementation

The two-child limit currently applies to:

- Universal Credit child elements
- Child Tax Credit child elements

By default, families can only claim these benefits for their first two children, with some exceptions:

1. Children born before April 6, 2017 are exempt from the limit
2. Multiple births (e.g., twins or triplets) where only one child would take the family over the two-child limit
3. Children adopted from local authority care
4. Children living with family or friends (kinship care)
5. Children born as a result of non-consensual conception

Our model currently simulates the date-based exemption (children born before April 2017).

## Policy Parameters

### Two-Child Limit Age Exemption

```
gov.contrib.two_child_limit.age_exemption
```

This parameter defines an age below which the two-child limit is waived for all children in a family. When set to a value greater than 0, if any child in the family is younger than this age threshold, the two-child limit does not apply to that family.

- **Default value**: 0 (no age-based exemption)
- **Unit**: years
- **Description**: Parents of children under this age are exempt from the two-child limit
- **Example usage**: Setting this to 3 would exempt all families with at least one child under 3 years old from the two-child limit

## How Age Exemption Works

When the `age_exemption` parameter is set to a value greater than 0:

1. The model checks if any child in the family is younger than the specified age
2. If true, the two-child limit does not apply to that family
3. All eligible children in the family receive the child element, regardless of birth order

This works across both benefit systems:
- For Universal Credit, all eligible children receive the UC child element
- For Child Tax Credit, all eligible children receive the CTC child element

## Effects on Family Income

The two-child limit can significantly impact family incomes, especially for larger families. For 2023-24:

- Universal Credit child element: approximately £287 per month per child (£3,444 annually)
- Child Tax Credit child element: approximately £3,070 annually per child

For a family with four children where all would otherwise be eligible, the two-child limit could reduce support by over £6,000 per year.

The age exemption reform allows policy analysts to model how exempting families with young children would affect household incomes and government expenditure.

## Examples

### Example 1: Family with children of different ages, no exemption

A family with four children aged 6, 4, 2, and 1 years old. With no age exemption (default policy):
- Only the first two children (aged 6 and 4) would receive the child element
- Total support is approximately £6,888 annually (UC) or £6,140 annually (CTC)

### Example 2: Same family with age exemption of 3 years

With an age exemption of 3 years:
- Because the family has children aged 2 and 1 (under 3), the limit is waived
- All four children would receive the child element
- Total support is approximately £13,776 annually (UC) or £12,280 annually (CTC)

## PolicyEngine Analysis Example

Below is a code example for analyzing the impact of implementing a 3-year age exemption to the two-child limit:

In [ ]:
import policyengine_uk
import pandas as pd
import matplotlib.pyplot as plt

# Set up the baseline simulation
baseline = policyengine_uk.Microsimulation()

# Set up the reform simulation with 3-year age exemption
reform = policyengine_uk.Microsimulation(
    reforms={
        "gov.contrib.two_child_limit.age_exemption": 3
    }
)

# Calculate the impact of the reform
impact = reform.calc("household_net_income") - baseline.calc("household_net_income")

# Identify families affected by the reform
has_children = baseline.calc("benunit_num_children") > 0
has_young_children = baseline.calc("benunit_has_children_under_age", age=3)
more_than_two_children = baseline.calc("benunit_num_children") > 2

affected_families = has_children & has_young_children & more_than_two_children

# Calculate average gain for affected families
avg_gain = (impact[affected_families] * baseline.calc("household_weight")[affected_families]).sum() / \
           baseline.calc("household_weight")[affected_families].sum()

print(f"Average annual gain for affected families: £{avg_gain:.2f}")

# Calculate total cost
total_cost = (impact * baseline.calc("household_weight")).sum()

print(f"Total annual cost of reform: £{total_cost/1e6:.1f} million")

## Modeling Considerations

When using these parameters for policy analysis:

1. The age exemption applies to the family as a whole - if any child is under the threshold, all children are exempt
2. The exemption works alongside existing exemptions (e.g., children born before April 2017)
3. The parameter can be varied to model different age thresholds and estimate impacts

This parameter enables modeling potential reforms to the two-child limit policy.